In [19]:
import numpy as np
import yaml
from model import UCCModel
import omegaconf
import hydra

import os
import sys

from time import time
from sklearn import metrics
from sklearn.cluster import KMeans, SpectralClustering

def cluster(estimator=None, name=None, data=None, labels=None):
	t0 = time()
	estimator.fit(data)
	homogeneity_score = metrics.homogeneity_score(labels, estimator.labels_)
	completeness_score = metrics.completeness_score(labels, estimator.labels_)
	v_measure_score = metrics.v_measure_score(labels, estimator.labels_)
	adjusted_rand_score = metrics.adjusted_rand_score(labels, estimator.labels_)
	adjusted_mutual_info_score = metrics.adjusted_mutual_info_score(labels,  estimator.labels_)

	print('%-9s\t%.2fs\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'% (name,
														(time() - t0),
														homogeneity_score,
														completeness_score,
														v_measure_score,
														adjusted_rand_score,
														adjusted_mutual_info_score))

	clustering_scores = np.array([homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score]).reshape((1,-1))
	return clustering_scores

In [20]:
output_path = ".\\outputs\\2024-03-20\\00-01-16"
model_name = "model_weights.h5"
# with open("outputs\\2024-03-18\\21-06-38\\.hydra\\config.yaml"), 'r') as file:
#     config = yaml.load(file)
cfg = omegaconf.OmegaConf.load(os.path.join(output_path, ".hydra\\config.yaml"))
print(cfg)
# cfg = hydra.utils.instantiate()
model = UCCModel(cfg)
model.build(input_shape=())
model.load_weights(filepath=os.path.join(output_path, model_name))

{'args': {'dataset': 'mnist', 'model_dir': 'saved_models/', 'model_name': 'mnist_ucc_1', 'num_instances': 32, 'ucc_start': 1, 'ucc_end': 4, 'batch_size': 20, 'num_samples_per_class': 5, 'learning_rate': 0.0001, 'num_bins': 11, 'num_features': 10, 'train_num_steps': 50000, 'val_num_steps': 200, 'save_interval': 1000, 'seed': 22}, 'model': {'num_channels': 1, 'kde_model': {'num_bins': 11, 'sigma': 0.1}, 'encoder': {'conv_input_channel': 1, 'conv_output_channel': 16, 'block1_output_channel': 32, 'block1_num_layer': 1, 'block2_output_channel': 64, 'block2_num_layer': 1, 'block3_output_channel': 128, 'block3_num_layer': 1, 'flatten_size': 6272, 'num_features': 10}, 'classification_model': {'fc1_output_size': 384, 'fc2_output_size': 192, 'dropout_rate': 0.0, 'num_classes': 4}, 'loss': {'alpha': 1}}}


In [21]:
splitted_dataset = np.load("../data/mnist/splitted_mnist_dataset.npz")
x_test = splitted_dataset["x_test"]/255
y_test = splitted_dataset["y_test"]

In [22]:
x_test = (x_test-np.mean(x_test, axis=(0,1,2)))/np.std(x_test, axis=(0,1,2))
x_test = np.expand_dims(x_test, axis=-1)

In [23]:
x_test.shape

(10000, 28, 28, 1)

In [24]:
features = model.extract_features(x_test)

In [25]:
from sklearn.cluster import KMeans, SpectralClustering
from cluster import cluster
k_estimator = KMeans(n_clusters=10)
s_estimator = SpectralClustering(n_clusters=10)
# pred = k_estimator.fit_predict(features)
print(cluster(
    estimator=k_estimator,
    data=features,
    labels=y_test
    ))

print(cluster(
    estimator=s_estimator,
    data=features,
    labels=y_test
    ))



None     	0.06s	0.430	0.472	0.450	0.254	0.449
[[0.43015482 0.47181526 0.45002293 0.25413705 0.44900488]]
None     	16.69s	0.447	0.466	0.456	0.274	0.455
[[0.44734614 0.46555313 0.45626808 0.27370583 0.45528726]]


In [18]:
print(np.unique(y_test))

[0 1 2 3 4 5 6 7 8 9]
